In [1]:
%matplotlib inline
%precision 16
import numpy
import matplotlib.pyplot as plt

Before you turn this problem in, make sure everything runs as expected. First, restart the kernel (in the menubar, select Kernel $\rightarrow$ Restart) and then run all cells (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says YOUR CODE HERE or "YOUR ANSWER HERE", as well as your name and collaborators below:

# HW 4:  Numerical Differentiation and Quadrature

## Question 1 - Fundamental Theorem of Calculus

**(a)** [10] Write a function that computes the integral of the derivative 

$$I[f] = \int^x_0 f'(s) ds$$

and returns the error from the expected calculation.  Use a second order accurate centered difference scheme and then a left-hand rule to compute the absolute error.  In this case the left-hand quadrature is

$$Q[f] = f(x_i) \Delta x$$

The function should take in $x$ values and the function $f(x)$ and return a vector of the **errors** for each value $x$ given.

In [ ]:
def int_diff_error(x, f):
    #x = numpy.linspace(0.0, 1.0, N+1)
    delta_x = x[1] - x[0]
    f_prime = numpy.empty(x.shape)
    # second order accurate centered difference 
    f_prime[0] = (f(x[1]) - f(x[0])) / delta_x 
    f_prime[-1] = (f(x[-1]) - f(x[-2])) / delta_x 
    for i in range(1, x.shape[0]-1):
        f_prime[i] = (f(x[i+1]) - f(x[i-1])) / (2.0 * delta_x)
    
    I = numpy.empty(x.shape)
    I[0] = f_prime[0] * delta_x + f(x[0])
    I[-1] = f_prime[-1] * delta_x
    for i in range(1, x.shape[0]):
        I[i] = I[i-1] + f_prime[i] * delta_x
    
    error = numpy.empty(x.shape)
    for i in range(0, x.shape[0]):
        error[i] = numpy.abs(I[i] - f(x[i]))

        
    #for i in range(0, N - 1):
        #I_hat += (f(x_hat[i + 1]) + f(x_hat[i])) * delta_x[j] / 2.0
    #error[i] = numpy.abs(I[i] - f(x[i]))
    return error

In [ ]:
f = lambda x: 1.0 / (1.0 + 100.0 * x**2)
x = numpy.linspace(0.0, 1.0, 500)
print numpy.max(int_diff_error(x, f))
numpy.testing.assert_allclose(int_diff_error(x, f), numpy.zeros(500), atol=1e-2)
print "Successful FTC evaluation."

**(b)** [10] Compute the same as above except reverse the order of the operations, i.e.

$$D[f] = \frac{\text{d}}{\text{d}x} \int^x_0 f(s) ds$$


In [ ]:
def diff_int_error(x, f):
    delta_x = x[1] - x[0]
    I = numpy.empty(x.shape)
    I[0] = f(x[0]) * delta_x
    for i in range(1, x.shape[0]):
        I[i] = I[i-1] + f(x[i]) * delta_x
    
    I_prime = numpy.empty(x.shape)
    I_prime[0] = (I[1] - I[0]) / delta_x
    I_prime[-1] = (I[-1] - I[-2]) / delta_x
    for i in range(1, x.shape[0]-1):
        I_prime[i] = (I[i+1] - I[i-1]) / (2.0 * delta_x)
    
    
    error = numpy.empty(x.shape)
    for i in range(0, x.shape[0]):
        error[i] = numpy.abs(I_prime[i] - f(x[i]))
        
    return error

In [ ]:
f = lambda x: 1.0 / (1.0 + 100.0 * x**2)
x = numpy.linspace(0.0, 1.0, 500)
print numpy.max(diff_int_error(x, f))
numpy.testing.assert_allclose(diff_int_error(x, f), numpy.zeros(500), atol=1e-2)
print "Successful FTC evaluation."

**(c)** [5] Plot the convergence rate for the number of partitions `N = [10,100,200,300,400,500]` on a `loglog` plot by computing the norm over the error given for each $N$ by using the `numpy.linalg.norm` command with `ord=2` vs. the $\Delta x$ used.  Theorize about what you observe regarding the order of convergence.

YOUR ANSWER HERE

In [2]:
f = lambda x: 1.0/(1.0+100.0*x**2)
N = numpy.array(30, 500, 100, dtype=int)
error = numpy.empty((N_range.shape[0],2))
delta_x = numpy.empty(N_range.shape)

for (i,N) in enumerate(N_range):
    x = numpy.linspace(0.0, 1.0 , N)
    delta_x[i] = x[1] - x[0]
    error[i,0] = numpy.linalg.norm(diff_int_error(x,f), ord=2)
    error[i,1] = numpy.linalg.norm(int_diff_error(x,f), ord=2)
error = numpy.array(error)
delta_x = numpy.array(delta_x)
fig = plt.figure()
fig.set_figwidth(2.0 * fig.get_figwidth())
fig.set_figheight(2.0 * fig.get_figheight())
error_type = ['diff_int','int_diff']

for i in range(error.shape[1]):
    axes = fig.add_subplot(2, 2, i+1)
    axes.loglog(delta_x, error[:, i], 'ko')
    
    order_C = lambda delta_x, error, order: numpy.exp(numpy.log(error) - order * numpy.log(delta_x))
    axes.loglog(delta_x, order_C(delta_x[0], error[0,i], 1.0) * delta_x**1.0, 'r--', label="1st order"
    axes.loglog(delta_x, order_C(delta_x[0], error[0,i], 2.0) * delta_x**2.0, 'b--', label="2nd order")
    axes.set_xlabel("$\Delta x")
    axes.set_ylabel("Error")
    axes.legend(loc=4)

    plt.show()

SyntaxError: invalid syntax (<ipython-input-2-3350ef939e5e>, line 24)

## Question 2 - Quadrature

Consider the scaled Fresnel integrals

$$I_c = \int^1_0 \frac{\cos x}{\sqrt{x}} dx~~~~\text{and}~~~~I_s = \int^1_0 \frac{\sin x}{\sqrt{x}} dx$$

which have values

$$I_c = \sqrt{2 \pi} \cdot C\left(\sqrt{\frac{2}{\pi}}\right) \approx 1.8090484758005438$$

$$I_s = \sqrt{2 \pi} \cdot S\left(\sqrt{\frac{2}{\pi}}\right) \approx 0.62053660344676231$$

where the functions $C(x)$ and $S(x)$ can be evaluated by `scipy.special.fresnel`.

**(a)** [2] Where do you imagine the problematic points of the integrals will be?

Solution: From the form of integrand, problematic point will be x = 0.

**(b)** [10] Write a function that computes both integrals using the  trapezoidal rule with N partitions of equal length "ignoring" the singularity at $x=0$ by setting integrands to 0.

In [26]:
def trap_1(N):
    Ic_grand = lambda x: numpy.cos(x) / numpy.sqrt(x)
    Is_grand = lambda x: numpy.sin(x) / numpy.sqrt(x)
    # num_partitions = N
    # x_hat = numpy.linspace(0.0, 1.0, N+1)
    x = numpy.linspace(0.0, 1.0, N+1)
    delta_x = x[1] - x[0]
    Ic = 0.0
    Is = 0.0
    for i in range(1, N):
        Ic += Ic_grand(x[i]) * delta_x
        Is += Is_grand(x[i]) * delta_x
    # to avoid mismatch, by setting integrand = 0 at x=0.
    Ic += Ic_grand(x[-1] + 0) * delta_x / 2.0
    Is += Is_grand(x[-1] + 0) * delta_x / 2.0
    
    return Is, Ic

In [ ]:
import scipy.special
Is, Ic = scipy.special.fresnel(numpy.sqrt(2.0 / numpy.pi))
Is *= numpy.sqrt(2.0 * numpy.pi)
Ic *= numpy.sqrt(2.0 * numpy.pi)
Is_hat, Ic_hat = trap_1(200)
error = numpy.abs(Is_hat - Is)
print "Error: %s" % error
numpy.testing.assert_allclose(error, 0.0, atol=1e-4)
print "Computed the integrals correctly using the trapezoidal rule only."    

**(c)** [10] A weighted Newton-Cotes quadrature rule is a modification of our version of Newton-Cotes quadrature except that we also multiply by a weighting function such that

$$\int^1_0 w(x) f(x) dx = \sum^{N}_{i=1} w_i f(x_i)$$

For the weight $x^{-1/2}$ we can show that 

$$\int^1_0 \frac{f(x)}{\sqrt{x}} dx \approx \frac{2}{3} (2 f(0) + f(1))$$

Combining this formula with the trapezoidal rule approach above, evaluate the integrals again by using the trapezoid rule except for the partition $[0,\Delta x]$ that involves the singularity use the above weighted Newton-Cotes rule.  Note that you will have to map the rule to the partition.

In [ ]:
def trap_wnc(N):
    Ic_grand = lambda x: numpy.cos(x) / numpy.sqrt(x)
    Is_grand = lambda x: numpy.sin(x) / numpy.sqrt(x)
    x = numpy.linspace(0.0, 1.0, N+1)
    delta_x = x[1] - x[0]
    Is = 0.0
    Ic = 0.0
    #delta_x = 1.0 / float(N)
    # by trapezoidal rule and a small interval using weighted NC rule. 
    for i in range(1, N):
        Ic += Ic_grand(x[i]) * delta_x
        Is += Is_grand(x[i]) * delta_x
    Ic += Ic_grand(x[-1]) * delta_x / 2.0
    Is += Is_grand(x[-1]) * delta_x / 2.0
    #Ic = Ic - Ic_grand(delta_x) * delta_x / 2.0 + 2.0 / 3.0 * (2.0 * numpy.cos(0.0) + numpy.cos(delta_x)) * delta_x / 2.0
    #Is = Is - Is_grand(delta_x) * delta_x / 2.0 + 2.0 / 3.0 * (2.0 * numpy.cos(0.0) + numpy.cos(delta_x)) * delta_x / 2.0
    
    return Is, Ic

In [ ]:
import scipy.special
Is, Ic = scipy.special.fresnel(numpy.sqrt(2.0 / numpy.pi))
Is *= numpy.sqrt(2.0 * numpy.pi)
Ic *= numpy.sqrt(2.0 * numpy.pi)
Is_hat, Ic_hat = trap_wnc(200)
error = numpy.abs(Is_hat - Is)
print "Error: %s" % error
numpy.testing.assert_allclose(error, 0.0, atol=1e-4)
print "Computed the integrals correctly using the trapezoidal and Newton-Cotes rules."    

**(d)** [10] Do a change of variables $x = t^2$ and evaluate each integral using the trapezoidal rule for the entire domain.

In [ ]:
def trap_transformed(N):
    # dx = 2t dt
    Ic_grand = lambda x: 2.0 * numpy.cos(x**2)
    Is_grand = lambda x: 2.0 * numpy.sin(x**2)
    # num_partitions = N
    # x_hat = numpy.linspace(0.0, 1.0, N+1)
    x = numpy.linspace(0.0, 1.0, N+1)
    delta_x = x[1] - x[0]
    Ic = 0.0
    Is = 0.0
    for i in range(1, N):
        Ic += Ic_grand(x[i]) * delta_x
        Is += Is_grand(x[i]) * delta_x
    # we have well-defined f(x[0]) for integrand
    Ic += (Ic_grand(x[-1]) + Ic_grand(x[0])) * delta_x / 2.0
    Is += (Is_grand(x[-1]) + Is_grand(x[0])) * delta_x / 2.0
    
    return Is, Ic

In [ ]:
import scipy.special
Is, Ic = scipy.special.fresnel(numpy.sqrt(2.0 / numpy.pi))
Is *= numpy.sqrt(2.0 * numpy.pi)
Ic *= numpy.sqrt(2.0 * numpy.pi)
Is_hat, Ic_hat = trap_transformed(200)
error = numpy.abs(Is_hat - Is)
print "Error: %s" % error
numpy.testing.assert_allclose(error, 0.0, atol=1e-5)
print "Computed the integrals correctly using the trapezoidal rule."    

**(e)** [10] Do the same as in part (c) but use 3-point Gauss-Legendre quadrature.

In [ ]:
def gauss_legendre_3(N):
    # update using part d
    Ic_grand = lambda x: 2.0 * numpy.cos(x**2)
    Is_grand = lambda x: 2.0 * numpy.sin(x**2)
    x = numpy.linspace(0.0, 1.0, N+1)
    delta_x = x[1] - x[0]
    Is = 0.0
    Ic = 0.0
    #delta_x = 1.0 / float(N) 
    # Compute Gauss-Legendre 3-point based on the lecture
    xi_map = lambda a,b,xi : (b - a) / 2.0 * xi + (a + b) / 2.0
    xi = [-numpy.sqrt(3.0 / 5.0), 0.0, numpy.sqrt(3.0 / 5.0)]
    w = [5.0 / 9.0, 8.0 / 9.0, 5.0 / 9.0]
    for i in range(0, N):
        for k in range(len(xi)):
            Ic += Ic_grand(xi_map(x[i], x[i+1], xi[k])) * w[k] 
            Is += Is_grand(xi_map(x[i], x[i+1], xi[k])) * w[k] 
            #I_hat += f(xi_map(x_hat[i], x_hat[i+1], xi[k])) * w[k]
    
    Ic *= delta_x / 2.0
    Is *= delta_x / 2.0
    
    #Ic = Ic - Ic_grand(delta_x + 0) * delta_x / 2.0 + 2.0 / 3.0 * (2.0 * numpy.cos(0.0) + numpy.cos(delta_x)) * delta_x / 2.0
    #Is = Is - Is_grand(delta_x + 0) * delta_x / 2.0 + 2.0 / 3.0 * (2.0 * numpy.cos(0.0) + numpy.cos(delta_x)) * delta_x / 2.0
    
    return Is, Ic

In [ ]:
import scipy.special
Is, Ic = scipy.special.fresnel(numpy.sqrt(2.0 / numpy.pi))
Is *= numpy.sqrt(2.0 * numpy.pi)
Ic *= numpy.sqrt(2.0 * numpy.pi)
Is_hat, Ic_hat = gauss_legendre_3(10)
error = numpy.abs(Is_hat - Is)
print "Error: %s" % error
numpy.testing.assert_allclose(error, 0.0, atol=1e-10)
print "Computed the integrals correctly using the trapezoidal and Newton-Cotes rules."    

**(f)** [5] Check the order of accuracy for each of the methods above and plot these on a `loglog` plot with appropriate reference lines to indicate the order of each.

**(g)** [3] Comment on the results of this question and the vast differences in convergence rates even between the two integrations (why is $I_c$ less accurate than $I_s$).

Solution: It is clear that the difficulty of integrating Ic derives from the singularity point at x=0, since cos(0) = 1. This makes the approximated value deviate from the true value comparing to sin(0) = 0. Therefore, Ic is less accurate than Is.

Meanwhile, by transformation of the form of the two functions, the singulariy point vanishes, and the function itself integrates well with the Guass-Legendre method.

## Question 3

We can often reformulate finite difference approximations as matrix-vector products.  For the following assume that the data considered are equi-spaced points $(x_i, y_i)$, i.e. $\Delta x$ is uniform.

**(a)** [10] Using a second order centered finite difference approximation to the second derivative and the appropriate forward and backward difference schemes at the edges of the domain find the matrix $D$ such that multiplying a vector of $y$ values would lead to a second order approximation of the derivative for the given data.  In other words for $y_i = f(x_i)$
$$
    f'(x) \approx D y.
$$

Solution: For uniform spaced points $\Delta$x, we have an n$\times$n matrix to satisfy second order centered finite difference approximation:
$$
    A = \frac{1}{\Delta x^2}\begin{bmatrix}
        2 & -5 & 4 & &-1 \\
      1 & -2 & 1 \\
      \cdots \\
      1 & &-2 & 1 \\
      \cdots &\cdots \\
      ~~~~~~~~~ &1 & -2 & 1 \\
      -1  &4 & -5 & 2
    \end{bmatrix}
$$

**(b)** [10] Write a function that takes in the number of data points $N$ and returns the matrix $D$.  Here assume we are on the interval $[-1, 1]$.  Note that inside your function 
$$
    \Delta x = \frac{2}{N - 1}
$$
to match the `linspace` command.

Beyond being convenient (once you construct the matrix you can apply it to any set of data) this operation is much faster than using loops.  *Hint:* The command `numpy.diag` may be extremely helpful.

In [28]:
def diff_matrix(N):
    D = numpy.zeros((N, N))
    delta_x = 2.0 / (N-1)
    diagonal = numpy.ones(N) / delta_x**2
    
    D += numpy.diag(diagonal * -2.0, 0)
    D += numpy.diag(diagonal[:-1], 1)
    D += numpy.diag(diagonal[:-1], -1)

    D[0, :4] = [2.0, -5.0, 4.0, -1.0]
    D[-1, -4:] = [-1.0, 4.0, -5.0, 2.0]
    D[0, :4] /= delta_x**2
    D[-1, -4:] /= delta_x**2
    
    return D

In [29]:
N = 200
x = numpy.linspace(-1, 1, N)
y = numpy.sin(x) * numpy.cos(x)
numpy.testing.assert_allclose(numpy.dot(diff_matrix(N), y), -4.0 * numpy.sin(x) * numpy.cos(x), rtol=1e-2)
print "Success!"

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Success!")? (<ipython-input-29-8589c42ea9b6>, line 5)

**(c)** [5] Suppose instead that we did not know the vector $f(x_i) = y_i$ but instead knew the value of second derivative at these points.  If we wanted to find the vector $y$ what kind of problem would we have to solve?  What is the continuous analog of this problem and what additional information would we need?

Solution: We have to solve a system of equations to get f(x$_i$) = y$_i$ using second derivatives.

In continuous analog, it is a system of ODE with boundary value and we have to know more about the boundary condition with boundary points or end points.